In [1]:
import pdfplumber
import pandas as pd
from src.widgets_helpers import file_selector_from_folder
import re
from src.helpers import save_as_csv
from src.parse_pdf import parse_pdf_text, separar_columnas_cc, extract_text_from_pdf_first_page, convert_transactions_to_df, convert_transactions_tarjeta_to_df, extract_text_from_pdf_rest_of_pages, separar_columnas_tajeta
from src.analyze import correct_and_merge_cc_t, add_initial_saldo_col

In [2]:
file = file_selector_from_folder("extractos/")

Dropdown(description='Archivos:', layout=Layout(width='50%'), options=(('Extracto_Febrero_2024.pdf', 'extracto…

In [3]:
text_cc = extract_text_from_pdf_first_page(file)

In [4]:
transacciones_cc = parse_pdf_text(text_cc,separar_columnas_cc)

In [5]:
df_cuenta = convert_transactions_to_df(transacciones_cc)

In [6]:
df_cuenta

,Fecha,Ref,Fecha Valor,Descripción,cargo,abono,Saldo
0,2024-02-02,846,2024-02-01,RECIBO PLATINUM,1285.52,0.00,"5.793,33"
1,2024-02-02,006,2024-02-01,PAGO BIZUM A NATALIA;GRANDA;AL,4.40,0.00,"5.788,93"
2,2024-02-07,134,2024-02-07,PAGO BIZUM A ROBERTO;PERINELLI,85.00,0.00,"5.703,93"
3,2024-02-10,201,2024-02-10,PAGO BIZUM A ANE;RAMSDEN;ARRIB,33.00,0.00,"5.670,93"
4,2024-02-10,322,2024-02-10,PAGO BIZUM A SAMIR RUBEN;AUYAN,70.80,0.00,"5.600,13"
5,2024-02-12,042,2024-02-12,PAGO BIZUM A DANIEL;TANO;LEAL,5.00,0.00,"5.595,13"
6,2024-02-12,505,2024-02-11,PAGO BIZUM A ANE;RAMSDEN;ARRIB,5.00,0.00,"5.590,13"
7,2024-02-12,662,2024-02-11,PAGO BIZUM A NATALIA;GRANDA;AL,31.50,0.00,"5.558,63"
8,2024-02-13,980,2024-02-13,RECIBO /SIMYO,23.43,0.00,"5.535,20"
9,2024-02-13,014,2024-02-11,CAJERO LA CAIXA OF.6738,80.00,0.00,"5.455,20"


In [7]:
text_t = extract_text_from_pdf_rest_of_pages(file)

In [8]:
transacciones_t = parse_pdf_text(text_t, separar_columnas_tajeta) 

In [9]:
df_tarjeta = convert_transactions_tarjeta_to_df(transacciones_t)

In [10]:
df_tarjeta

,Fecha,Descripción,cargo,abono
0,2024-01-31,"CRUNCH IB RICO, LAS PALMAS",2.50,0
1,2024-01-31,"HD PORTUGAL, 35010",6.57,0
2,2024-01-31,"COM. USO REDES INTERNACION., SAN 0,60",0.56,0
3,2024-01-31,"CHATGPT SUBSCRIPTION, SAN FRANCI 20,00",18.51,0
4,2024-02-01,"PAYPAL *GLOVO, 35314369001",29.79,0
...,...,...,...,...
118,2024-02-27,"COM. USO REDES INTERNACION., 402-935-7733",0.14,0
119,2024-02-27,"PAYPAL *MEDIUM.COM, 402-935-7733",4.82,0
120,2024-02-28,"EL MOMENTO, LAS PALMAS",3.60,0
121,2024-02-04,"RESTAURANTE LAS NASAS, AGAETE",24.88,0


In [11]:
df = correct_and_merge_cc_t(df_cuenta, df_tarjeta)

In [12]:
df

,Fecha,Descripción,cargo,abono
0,2024-01-31,"CHATGPT SUBSCRIPTION, SAN FRANCI 20,00",18.51,0.0
1,2024-01-31,"COM. USO REDES INTERNACION., SAN 0,60",0.56,0.0
2,2024-01-31,"HD PORTUGAL, 35010",6.57,0.0
3,2024-01-31,"CRUNCH IB RICO, LAS PALMAS",2.50,0.0
4,2024-02-01,"HD PORTUGAL, 35010",1.78,0.0
...,...,...,...,...
139,2024-02-27,"PAYPAL *MEDIUM.COM, 402-935-7733",4.82,0.0
140,2024-02-28,"EL MOMENTO, LAS PALMAS",3.60,0.0
141,2024-02-28,PAGO BIZUM A SAMIR RUBEN;AUYAN,6.00,0.0
142,2024-02-28,PAGO BIZUM A NURIA;MORA;BETANC,4.00,0.0


In [13]:
df = add_initial_saldo_col(df, df_cuenta)

In [14]:
df

,Fecha,Descripción,cargo,abono,Saldo
0,2024-01-31,"CHATGPT SUBSCRIPTION, SAN FRANCI 20,00",18.51,0.0,5774.82
1,2024-01-31,"COM. USO REDES INTERNACION., SAN 0,60",0.56,0.0,5792.77
2,2024-01-31,"HD PORTUGAL, 35010",6.57,0.0,5786.76
3,2024-01-31,"CRUNCH IB RICO, LAS PALMAS",2.50,0.0,5790.83
4,2024-02-01,"HD PORTUGAL, 35010",1.78,0.0,5791.55
...,...,...,...,...,...
139,2024-02-27,"PAYPAL *MEDIUM.COM, 402-935-7733",4.82,0.0,5788.51
140,2024-02-28,"EL MOMENTO, LAS PALMAS",3.60,0.0,5789.73
141,2024-02-28,PAGO BIZUM A SAMIR RUBEN;AUYAN,6.00,0.0,5787.33
142,2024-02-28,PAGO BIZUM A NURIA;MORA;BETANC,4.00,0.0,5789.33


In [15]:
save_as_csv(df,file)

In [16]:
file

'extractos/Extracto_Febrero_2024.pdf'